In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('2018_Incident_Report.csv')

In [9]:
df.head()
df.info()
#df['Primary Key'] = df['Primary Key'].astype('str')
#df['Incident Number'] = df['Incident Number'].astype('str')
#df['Report Type'] = df['Report Type'].astype('str') 
#df['Report Type Description'] = df['Report Type Description'].astype('str') 
#df['Incident Status Code'] = df['Incident Status Code'].astype('str') 
#df['Incident Status Description'] = df['Incident Status Description'].astype('str')
#df['Investigation Status'] = df['Investigation Status'].astype('str')
df['Incident Occurred'] = df['Incident Occurred'].astype('datetime64')
df['Incident Reported'] = df['Incident Reported'].astype('datetime64')
df['Incident_Time_Lapse'] = (df['Incident Reported']-df['Incident Occurred'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85555 entries, 0 to 85554
Data columns (total 31 columns):
Primary Key                    85555 non-null object
Incident Number                85555 non-null object
Report Type                    85555 non-null object
Report Type Description        85555 non-null object
Incident Status Code           85555 non-null object
Incident Status Description    85555 non-null object
Investigation Status           85555 non-null object
Incident Occurred              85555 non-null datetime64[ns]
Incident Reported              85555 non-null datetime64[ns]
Incident Location              85269 non-null object
Zip Code                       45823 non-null float64
RPA                            44458 non-null float64
Zone                           44458 non-null float64
Location Code                  85555 non-null int64
Location Description           85555 non-null object
Offense Number                 85543 non-null float64
Offense NIBRS           

In [11]:
df['Incident_Time_Lapse'].head()

0   05:36:00
1   13:30:00
2   13:47:00
3   01:04:00
4   02:07:00
Name: Incident_Time_Lapse, dtype: timedelta64[ns]

In [13]:
df.groupby('Victim Race').nunique()

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Latitude,Longitude,Mapped Location,Incident_Time_Lapse
Victim Race,,,,,,,,,,,,,,,,,,,,,
A,678,625,5,4,5,5,2,601,625,482,...,4,4,2,1,3,3,164,189,444,483
B,23052,20598,5,4,7,7,2,15828,19963,5848,...,6,6,3,1,3,3,326,374,4998,4457
H,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
I,67,61,4,4,4,4,2,60,61,58,...,1,1,2,1,3,2,37,44,56,53
O,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
P,43,37,4,4,4,4,2,37,37,36,...,2,2,2,1,3,2,29,29,36,35
U,3887,3509,5,4,7,7,2,3329,3473,2220,...,8,8,3,1,3,3,304,339,2080,1070
W,33484,29796,5,4,7,7,2,19900,28368,9183,...,7,5,3,1,3,3,378,453,7353,6890


In [26]:
df['Incident_Time_Lapse'].mean()

Timedelta('4 days 23:45:02.797498')